# User Takeover
Exploring when the user accounts that were bought were taken over, specifically to get a more accurate representation of what the bot behavior actually looks like

In [1]:
import pandas as pd
import numpy as np

In [2]:
users1 = pd.read_csv("../data/users1.csv")
users2 = pd.read_csv("../data/users2.csv")
users3 = pd.read_csv("../data/users3.csv")

In [6]:
users = pd.concat([users1, users2, users3])
users.shape

(5241, 10)

In [4]:
tweets1 = pd.read_csv("../data/tweets1.csv", low_memory=False)
tweets2 = pd.read_csv("../data/tweets2.csv", low_memory=False)
tweets3 = pd.read_csv("../data/tweets3_1.csv", low_memory=False)
tweets4 = pd.read_csv("../data/tweets3_2.csv", low_memory=False)
tweets5 = pd.read_csv("../data/tweets3_3.csv", low_memory=False)

In [7]:
tweets = pd.concat([tweets1, tweets2, tweets3, tweets4, tweets5])
tweets.shape

(13847731, 31)

The idea here is that many of the accounts have a point where the user activity completely changes, most notably the language. We want to find the point where these accounts change language, mostly so we can get better statistics about their behavior.

Some caveats to keep in mind:
- It's possible that some accounts are multi-lingual
- It's possible the account language == tweet language

In [7]:
mismatched_users = tweets[tweets['tweet_language'] != tweets['account_language']]

In [8]:
mismatched_userid = mismatched_users.groupby('userid').count().index.to_list()

In [9]:
mismatched_user_acc = users[users['userid'].isin(mismatched_userid)]
mismatched_user_acc.shape, users.shape

((5179, 10), (5241, 10))

Let's try a small case study on some of these taken over accounts

Example username = coolchefff666

Interesting example, as the account was created a long time ago, in 2009.

Lets look at:
- Tweeting activity before and after takeover
        - Variance in posting activity by day
        - Variance in posting activity by hour

In [30]:
case_study_name = 'coolchefff666'
case_study_acc = mismatched_users[mismatched_users['user_screen_name'] == 'coolchefff666'].drop([
    'hashtags', 'reply_count', 'quote_count', 'like_count', 'reply_count', 'retweet_count', 'urls', 'user_reported_location', 'user_profile_description', 'user_profile_url',
    'user_mentions', 'poll_choices', 'in_reply_to_userid', 'in_reply_to_tweetid', 'quoted_tweet_tweetid', 'is_retweet', 'retweet_userid', 'retweet_tweetid', 'latitude', 'longitude'
], axis=1)
case_study_acc

,tweetid,userid,user_display_name,user_screen_name,follower_count,following_count,account_creation_date,account_language,tweet_language,tweet_text,tweet_time,tweet_client_name
43,5962215854,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,NaN,"Buffalo Wild Wings, Inc. Announces Third Quart...",2009-11-23 01:32,web
44,7161955469,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,NaN,Frisch's Reports Higher Earnings in First Quar...,2009-12-29 15:30,web
45,6690127535,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,NaN,"Chocolate-infused spirits with kick, not sugar...",2009-12-15 07:56,web
46,7976854184,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,NaN,Jamba Juice Lays Down New Roots in New York Ci...,2010-01-20 05:44,web
47,6363903387,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,NaN,McDonald's Third Quarter Earnings Rise on Stro...,2009-12-05 07:25,web
...,...,...,...,...,...,...,...,...,...,...,...,...
2753129,1128173598982561794,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,tl,Mini Vietnamese Banh Mi Meatballs #vietnamesem...,2019-05-14 05:42,Twitter Web Client
2826417,1129549352806936576,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,und,https://t.co/z6tvKvbUi6,2019-05-18 00:48,Twitter Web Client
2844642,1127008910504648704,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,en,How to Make Sous Vide Duck Confit https://t.co...,2019-05-11 00:33,Twitter Web Client
2855284,1129923774990704640,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,en,Instant-Pot Kheer Recipe (Indian Rice Pudding)...,2019-05-19 01:36,Twitter Web Client


In [43]:
case_study_acc['tweet_time'] = pd.to_datetime(case_study_acc['tweet_time'])
case_study_acc = case_study_acc.sort_values(by='tweet_time').reset_index()
case_study_acc.tail(50)

,index,tweetid,userid,user_display_name,user_screen_name,follower_count,following_count,account_creation_date,account_language,tweet_language,tweet_text,tweet_time,tweet_client_name
726308,2580584,811162999528521728,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,en,#SuperChef - #Fish Stuffed #Bajji https://t.co...,2016-12-20 10:55:00,SocialOomph
726309,279066,811164605854584832,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,en,#Pasta With Simple Tomato Sauce #Recipe https:...,2016-12-20 11:01:00,SocialOomph
726310,679819,811166014478188544,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,en,How to #Caramelize Creme #Brulee https://t.co...,2016-12-20 11:07:00,SocialOomph
726311,608448,811167546531520512,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,en,#Russian #Chicken Salad #Recipe #Chef Baba Coo...,2016-12-20 11:13:00,SocialOomph
726312,1922298,811169060557819905,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,en,#Tuna #Recipe Home #Cooking from #ChefRicardo ...,2016-12-20 11:19:00,SocialOomph
726313,36660,811170599141855233,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,en,#ChefGaby Dalkin Shares #Recipes for Women on ...,2016-12-20 11:25:00,SocialOomph
726314,3932696,811172048869425152,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,en,Pork Neck Bone #Spaghetti Sauce #Recipe https:...,2016-12-20 11:31:00,SocialOomph
726315,2047414,1111425898983514117,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,en,RT @Tpl_Emmy: Who's still your favorite female...,2019-03-29 00:32:00,Twitter Web Client
726316,2576899,1118681670180167685,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,zh,Online conference 2019 https://t.co/hRSCq8xm1o...,2019-04-18 01:04:00,Twitter Web Client
726317,1813809,1118720333165416448,54107005,Cool Chef,coolchefff666,62016,63183,2009-07-06,zh-cn,en,Instant-Pot Kheer #Recipe (Indian Rice Pudding...,2019-04-18 03:38:00,Twitter Web Client


In [44]:
case_study_acc.iloc[726357].tweet_text

'闹剧被拆穿无数的郭文贵，最近编造谎言确实不太走心，不仅创意是烂的可以，内容也是极为可笑，空剩下一副蹭热点的激情，但苦于军师不足，结果漏洞百出，五月的黑色依旧延续，估计郭文贵的六月同样也是黑色。 https://t.co/N54VNVN9s1'

**Translation:**

*The farce has been dismantled by countless Guo Wengui. Recently, it is true that the lie is not very deliberate. Not only is the creativity bad, but the content is also extremely ridiculous. There is only a pair of hot spots left. However, due to insufficient military personnel, the result is full of loopholes. The black continues, and it is estimated that Guo Wengui's June is also black.*


Guo Wengui is a Chinese billionaire who has previously accused the Communist Chinese Party of corruption, and is currently exiled. He fled from China in 2014, due to being charged with accusations of bribery, fraud, money laundering and rape - all of which he denies. He has previously worked with Steve Bannon to investigate the CCP and has spent over 100 million USD on that campaign, and is now being investigated by the FBI.

These account appear to be focusing on using retweets from multiple sources that can amass followers,
